# 06 - Report & Summary
Roll up refusal, toxicity, and topic signals into a compact report artifact.

**Goals**
- Load processed imitation + Perspective bundles (including the more_refuse variant).
- Derive refusal using true_rate when available and summarize refusal/toxicity metrics.
- Surface topic highlights from migrated LDA assets and persist a lightweight report.

In [ ]:
from pathlib import Path
import json
import pickle
from typing import Dict, List, Optional, Sequence, Union

import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from utils.data_io import load_df_list_pickle, flatten_conversation_bundles, describe_bundle

In [ ]:
# Paths and toggles
PROJECT_ROOT = Path.cwd()
ASSETS_RAW = PROJECT_ROOT / "assets" / "raw"
ASSETS_PROCESSED = PROJECT_ROOT / "assets" / "processed"
ASSETS_TOPICS = ASSETS_PROCESSED / "topics"
REPORT_DIR = ASSETS_PROCESSED / "report"

IMMITATION_PATH = ASSETS_PROCESSED / "combat_threads_with_imitation.pkl"
PERSPECTIVE_PATH = ASSETS_PROCESSED / "combat_threads_with_perspective.pkl"
PERSPECTIVE_LIST_PATH = ASSETS_PROCESSED / "combat_threads_with_perspective_list.pkl"
PERSPECTIVE_MORE_REFUSE_PATH = ASSETS_PROCESSED / "combat_threads_with_perspective_list_more_refuse_cleaned.pkl"

LDA_TFIDF_REF = ASSETS_TOPICS / "lda_results_tfidf_ref.pkl"
LDA_TFIDF_ACC = ASSETS_TOPICS / "lda_results_tfidf_acc.pkl"
LDA_COUNT_REF = ASSETS_TOPICS / "lda_results_count_ref.pkl"
LDA_COUNT_ACC = ASSETS_TOPICS / "lda_results_count_acc.pkl"

SOURCE_MODE = "more_refuse"  # options: more_refuse, perspective_list, base
HIGH_TOXICITY = 0.5  # label a turn as toxic when Perspective TOXICITY >= this
REFUSAL_CONVERSATION_THRESHOLD = 0.1  # bucket conversations by refusal share
REFUSAL_TRUE_RATE_THRESHOLD = 0.67  # more_refuse bundle marks refusal when true_rate falls below this share

REPORT_DIR.mkdir(parents=True, exist_ok=True)

ASSETS_PROCESSED


### Asset manifest
List primary inputs and migrated topic assets used to generate the report.

In [ ]:
manifest = [
    {
        'role': 'input',
        'path': IMMITATION_PATH,
        'note': 'Imitation bundle with imm_1 + imm_1_check (source: Raja/Convo/combat_df_list_imms_1_full.pkl).',
    },
    {
        'role': 'input',
        'path': PERSPECTIVE_PATH,
        'note': 'Imitation + raw Perspective dicts (source: Raja/revised_convo/combat_df_list_imms_1_full_perspective.pkl).',
    },
    {
        'role': 'variant_optional',
        'path': PERSPECTIVE_LIST_PATH,
        'note': 'Perspective list-encoded scores (source: Raja/revised_convo/combat_df_list_imms_1_full_perspective_list.pkl).',
    },
    {
        'role': 'variant_optional',
        'path': PERSPECTIVE_MORE_REFUSE_PATH,
        'note': 'Perspective list + refuse_add/true_rate heuristics (source: Raja/revised_convo/combat_df_list_imms_1_full_perspective_list_moreRefuse_refuseCleaned.pkl).',
    },
    {
        'role': 'topics_optional',
        'path': LDA_TFIDF_REF,
        'note': 'TF-IDF LDA on refusal conversations (migrated from Raja/revised_convo/lda_results_tfidf_ref.pkl).',
    },
    {
        'role': 'topics_optional',
        'path': LDA_TFIDF_ACC,
        'note': 'TF-IDF LDA on acceptance conversations (migrated from Raja/revised_convo/lda_results_tfidf_acc.pkl).',
    },
    {
        'role': 'topics_optional',
        'path': LDA_COUNT_REF,
        'note': 'Count-vector LDA on refusal conversations (migrated from Raja/revised_convo/lda_results_count_ref.pkl).',
    },
    {
        'role': 'topics_optional',
        'path': LDA_COUNT_ACC,
        'note': 'Count-vector LDA on acceptance conversations (migrated from Raja/revised_convo/lda_results_count_acc.pkl).',
    },
    {
        'role': 'output_optional',
        'path': REPORT_DIR / f"report_{SOURCE_MODE}.json",
        'note': 'Summary JSON written by this notebook (metrics + topic highlights).',
    },
]

pd.DataFrame(manifest)


### Load bundle and derive refusal
Use true_rate when available (more_refuse) otherwise fallback to imm_1_check.

In [ ]:
def resolve_source(mode: str):
    if mode == 'more_refuse' and PERSPECTIVE_MORE_REFUSE_PATH.exists():
        return PERSPECTIVE_MORE_REFUSE_PATH
    if mode == 'perspective_list' and PERSPECTIVE_LIST_PATH.exists():
        return PERSPECTIVE_LIST_PATH
    return PERSPECTIVE_PATH

def derive_is_refusal(frame: pd.DataFrame, mode: str) -> pd.Series:
    if mode == 'more_refuse' and 'true_rate' in frame.columns:
        return frame['true_rate'].fillna(0) < REFUSAL_TRUE_RATE_THRESHOLD
    return ~frame['imm_1_check'].astype(bool)

SOURCE_PATH = resolve_source(SOURCE_MODE)
if not SOURCE_PATH.exists():
    raise FileNotFoundError(f'Bundle not found: {SOURCE_PATH}')

bundle = load_df_list_pickle(SOURCE_PATH)
print(f'using source: {SOURCE_PATH.name}')
print('bundle summary:', describe_bundle(bundle))

flat = flatten_conversation_bundles(bundle)
flat['is_refusal'] = derive_is_refusal(flat, SOURCE_MODE)
print('rows', len(flat))
print('columns', flat.columns.tolist())
flat.head()

### Expand Perspective scores
Normalize Perspective attributes (dict or list) into flat columns for downstream metrics.

In [ ]:
PERSPECTIVE_ATTRIBUTES = [
    'AFFINITY_EXPERIMENTAL',
    'COMPASSION_EXPERIMENTAL',
    'CURIOSITY_EXPERIMENTAL',
    'IDENTITY_ATTACK',
    'IDENTITY_ATTACK_EXPERIMENTAL',
    'INSULT',
    'INSULT_EXPERIMENTAL',
    'NUANCE_EXPERIMENTAL',
    'PERSONAL_STORY_EXPERIMENTAL',
    'PROFANITY',
    'PROFANITY_EXPERIMENTAL',
    'REASONING_EXPERIMENTAL',
    'RESPECT_EXPERIMENTAL',
    'SEVERE_TOXICITY',
    'SEVERE_TOXICITY_EXPERIMENTAL',
    'SEXUALLY_EXPLICIT',
    'THREAT',
    'THREAT_EXPERIMENTAL',
    'TOXICITY',
    'TOXICITY_EXPERIMENTAL',
]

def extract_summary_scores(entry: Optional[dict], attributes: Sequence[str] = PERSPECTIVE_ATTRIBUTES) -> Dict[str, Optional[float]]:
    scores: Dict[str, Optional[float]] = {}
    for attr in attributes:
        key = f'persp_{attr.lower()}'
        value = None
        if isinstance(entry, dict):
            value = entry.get(attr, {}).get('summaryScore', {}).get('value')
        scores[key] = value
    span_end = None
    if isinstance(entry, dict):
        spans = entry.get(attributes[0], {}).get('spanScores', [])
        if spans:
            span_end = spans[0].get('end')
    scores['persp_span_end'] = span_end
    return scores

def perspective_row_to_dict(row: pd.Series) -> Dict[str, Optional[float]]:
    ls = row.get('perspective_ls')
    if isinstance(ls, (list, tuple)) and len(ls) >= len(PERSPECTIVE_ATTRIBUTES) + 1:
        scores = {f'persp_{attr.lower()}': val for attr, val in zip(PERSPECTIVE_ATTRIBUTES, ls[1:])}
        scores['persp_span_end'] = ls[0]
        return scores
    return extract_summary_scores(row.get('perspective'))

flat_reset = flat.reset_index(drop=True)
score_frame = pd.DataFrame(flat_reset.apply(perspective_row_to_dict, axis=1).tolist())
analysis_df = pd.concat([flat_reset, score_frame], axis=1)
analysis_df['is_toxic'] = analysis_df['persp_toxicity'].fillna(0) >= HIGH_TOXICITY
analysis_df.head()

### Summary metrics
Refusal shares and toxicity breakdowns (overall vs refusal vs acceptance).

In [ ]:
refusal_rate = analysis_df['is_refusal'].mean()
convo_refusal = analysis_df.groupby('conversation_idx')['is_refusal'].mean()
refusal_convo_share = (convo_refusal > REFUSAL_CONVERSATION_THRESHOLD).mean()

toxicity_rate = analysis_df['is_toxic'].mean()
toxicity_refuse = analysis_df.loc[analysis_df['is_refusal'], 'is_toxic'].mean()
toxicity_accept = analysis_df.loc[~analysis_df['is_refusal'], 'is_toxic'].mean()

summary_metrics = {
    'rows': len(analysis_df),
    'refusal_rate': refusal_rate,
    'refusal_conversation_share': refusal_convo_share,
    'toxicity_rate': toxicity_rate,
    'toxicity_rate_refusals': toxicity_refuse,
    'toxicity_rate_acceptances': toxicity_accept,
}

pd.Series(summary_metrics)

In [ ]:
examples = (
    analysis_df.sort_values('persp_toxicity', ascending=False)
    [['conversation_idx', 'text', 'imm_1', 'is_refusal', 'persp_toxicity']]
    .head(5)
)
examples

### Topic highlights
Load best runs from migrated LDA assets and surface top terms per split.

In [ ]:
def load_lda_results(path: Path) -> Optional[List[Dict]]:
    if not path.exists():
        return None
    with path.open('rb') as fp:
        return pickle.load(fp)

def choose_best(runs: Optional[List[Dict]]) -> Optional[Dict]:
    if not runs:
        return None
    return max(
        runs,
        key=lambda r: (
            r.get('coherence') if r.get('coherence') is not None else -1,
            -(r.get('perplexity') or 0),
        ),
    )

def topics_frame(entry: Dict, label: str) -> pd.DataFrame:
    return pd.DataFrame(
        {
            'topic': range(len(entry.get('topics', []))),
            'top_terms': [' '.join(words) for words in entry.get('topics', [])],
            'n_topics': entry.get('n_topics'),
            'vectorizer': entry.get('vectorizer').__class__.__name__ if entry.get('vectorizer') else None,
            'asset': label,
        }
    )

lda_assets = {
    'tfidf_ref': LDA_TFIDF_REF,
    'tfidf_acc': LDA_TFIDF_ACC,
    'count_ref': LDA_COUNT_REF,
    'count_acc': LDA_COUNT_ACC,
}

topic_tables: List[pd.DataFrame] = []
best_topics: Dict[str, Dict] = {}
for label, path in lda_assets.items():
    runs = load_lda_results(path)
    best = choose_best(runs)
    if best:
        topic_tables.append(topics_frame(best, label))
        best_topics[label] = {
            'n_topics': best.get('n_topics'),
            'top_terms': [' '.join(words) for words in best.get('topics', [])],
        }

pd.concat(topic_tables, ignore_index=True) if topic_tables else 'No topic assets loaded.'

### Export report snapshot
Persist key metrics and topic highlights to assets/processed/report/.

In [ ]:
REPORT_PATH = REPORT_DIR / f'report_{SOURCE_MODE}.json'
report_payload = {
    'source_mode': SOURCE_MODE,
    'source_path': str(SOURCE_PATH),
    'refusal_true_rate_threshold': REFUSAL_TRUE_RATE_THRESHOLD if SOURCE_MODE == 'more_refuse' else None,
    'high_toxicity_threshold': HIGH_TOXICITY,
    'refusal_conversation_threshold': REFUSAL_CONVERSATION_THRESHOLD,
    'metrics': summary_metrics,
    'topic_highlights': best_topics,
}

with REPORT_PATH.open('w') as fp:
    json.dump(report_payload, fp, indent=2)

REPORT_PATH